In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import mysql.connector
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from user_agent import generate_user_agent, generate_navigator
from selenium.common.exceptions import UnexpectedAlertPresentException
import requests
import datetime

import mysql
import matplotlib.pyplot as plt
import cv2 
import os
import glob
import joblib

from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import ResNet50_Weights
import  torch.nn.functional as F


import MeCab
from konlpy.tag import Okt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import sys
sys.path.append('/Users/anhyojun/WorkSpace/KDT2/MyModule')
from KDTModule import *

In [2]:
torch.manual_seed(1)

In [3]:
movie = pd.read_csv('movieDF.csv')

In [4]:
story = movie['story']
grade_dict = {'전체 관람가' : 0, '12세이상관람가' : 1, '15세이상관람가' : 2, '청소년 관람불가' : 3}
grade = movie['grade'].replace(grade_dict)

In [5]:
text_list = list(story.values)
label_list = list(grade.values)

In [6]:
text_list[:5], label_list[:5]

(['LA 경찰국 강력계의 빈센트 한나(알 파치노) 반장은 두 번의 이혼 경력과 순탄치 않은 세 번째 결혼생활로 불안하고 우울한 일상을 이어간다. 어느 날 특급 우편 차량을 급습하는 사건이 일어나고 호송 요원 세 명이 무참히 살해되자 한나는 특유의 예리한 감으로 닐 맥컬리(로버트 드 니로)의 존재를 찾아낸다. 자신의 팀원들을 가족처럼 보살피는 빈틈없는 프로 범죄자 닐은 자신을 쫓는 한나를 비웃듯 따돌리고 닐의 용의주도하고 프로페셔널한 면모에 한나는 닐에 대한 관심과 승부욕이 생기기 시작하는데… 액션의 전설이 다시 부활한다!',
  '미국 4대 항공사인 엔디버 항공사의 사장이자 재력가인 톰 멀론은 성공의 상징이며 언제나 매스컴의 주목을 받고 있는 유명인사다. 뉴욕시의 과학발명품 경연대회가 있던 날 톰과 케이트의 아들인 숀이 납치된다. 범인들은 숀의 몸값으로 2백만 달러를 요구한다. FBI에서 사건을 맡지만 톰은 아들을 구하겠다는 일념으로 돈을 들고 약속 장소로 나간다. 1차 작전이 실패하고 극도로 불안한 톰은 방송에 출연해 유괴범을 잡는 사람에게 4백만 달러의 현상금을 지불하겠다는 선전포고를 한다.',
  '에드워드는 망해가는 회사를 사들여 나누어서 비싸게 파는 일을 하는 부유하고도 냉정한 사업가다. 사업차 로스엔젤레스로 간 그는 그 곳에서 비비안이라는 창녀를 만나 하룻밤을 지낸다. 그는 영화를 보면서 웃거나 목욕을 하며 노래를 부르는 그녀의 모습을 보고 하루가 아니라 한 주 동안 자신과 같이 지낼 것을 제안한다. 그리고 비비안은 에드워드와 중요한 자리에 참석하기 위해 완전히 딴 사람으로 변신한다. 계약 기간 동안 같이 지내게 된 리차드와 비비안은 서로에게 호감을 느낀다. 에드워드는 비비안과 함께 잠시 일에서 벗어나 한가로운 시간을 보내며 회사를 나누어 팔기만 하는 자신의 일에 대해 다시 한번 생각하게 된다.  계약했던 기간이 다 지나고 에드워드는 그녀를 계속 만나고 싶어하지만 비비안은 숨겨진 여자로 관계를 지속시키고 싶어하지 않는다. 결국 그녀는 그를 떠나는데

In [7]:
# 훈련, 검증, 테스트 분리
train_texts, test_texts, train_labels, test_labels = train_test_split(text_list, label_list, test_size=0.2,
                                                                      stratify=label_list, random_state=42)
train_texts, valid_texts, train_labels, valid_labels = train_test_split(train_texts, train_labels, test_size=0.2,
                                                                        stratify=train_labels, random_state=42)

print(f"[train] : {len(train_texts)}")
print(f"[valid] : {len(valid_texts)}")
print(f"[test] : {len(test_texts)}")

[train] : 13824
[valid] : 3457
[test] : 4321


'깨소금이라는 별명의 주리와 옥떨메라는 별명의 공주는 단짝으로 학교에서 명물로 통한다. 교장선생님과의 애교섞인 투쟁, 담임선생님과의 깜찍한 설전, 아담한 남학생 슬기와의 골탕먹이기 작전, 남학생 삼수와의 풋풋한 사랑, 언니와 미술선생님 간의 신나는 중매작전이 이루어진다.  그러던 어느 날, 깨소금과 옥떨메는 사소한 일로 다투게 되고 서먹서먹하게 된다. 개학식날 옥떨메의 모습이 보이지 않자 깨소금은 잘못을 뉘우치고 친구들과 옥떨메 찾기에 나선다. 가정형편으로 학교를 그만두고 계란장사를 하는 옥떨메를 발견하고 친구들이 옥떨메돕기운동에 나서 온통 계란소동이 나는데...'

In [61]:
sentence = request.form['tweet']

class TextModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_list, act_func, model, num_layers=1):
        super().__init__()
        # 입력층 (LSTM)
        if model == 'lstm':
            self.lstm_layer = nn.LSTM(input_size, hidden_list[0], num_layers, batch_first=True)
        elif model == 'rnn':
            self.rnn_layer = nn.RNN(input_size, hidden_list[0], num_layers, batch_first=True)
        elif model == 'gru':
            self.gru_layer = nn.GRU(input_size, hidden_list[0], num_layers, batch_first=True)
        # 은닉층
        self.hidden_layer_list = nn.ModuleList()
        for i in range(len(hidden_list)-1):
            self.hidden_layer_list.append(nn.Linear(hidden_list[i], hidden_list[i+1]))
        # 출력층
        self.output_layer = nn.Linear(hidden_list[-1], output_size)

        self.act_func = act_func
        self.dropout = nn.Dropout(0.5)
        self.model = model
        
    def forward(self, x):
        # 입력층
        if self.model == 'lstm':
            lstm_out, (hn, cn) = self.lstm_layer(x) # lstm_out : 모든 타입스텝 출력
            x = lstm_out[:, -1, :] # 마지막 타입스텝 출력
        elif self.model == 'rnn':
            rnn_out, hn = self.rnn_layer(x) # rnn_out : 모든 타입스텝 출력
            x = rnn_out[:, -1, :] # 마지막 타입스텝 출력
        elif self.model == 'gru':
            gru_out, hn = self.gru_layer(x) # gru_out : 모든 타입스텝 출력
            x = gru_out[:, -1, :] # 마지막 타입스텝 출력
        # 은닉층
        for layer in self.hidden_layer_list:
            x = layer(x)
            x = self.act_func(x)
            x = self.dropout(x)
        # 출력층
        return self.output_layer(x) # 로짓값

with open('movie_stopword.txt', 'r', encoding='utf-8') as f:
    stopwords = [i.strip() for i in f.readlines()]

token = Okt().morphs(sentence)
clean_token = remove_punctuation(token)
clean_token = remove_stopwords(clean_token, stopwords)
clean_text = ' '.join(clean_token)
textDF = pd.DataFrame({'clean_text' : [clean_text]})

loaded_vectorizer = joblib.load('tfidfvectorizer.pkl')
text_vector = loaded_vectorizer.transform(textDF['clean_text']).toarray()
text_vectorDF = pd.DataFrame(text_vector)

best_model = TextModel(input_size = 20000, output_size = 1, hidden_list = [100],
                       act_func=F.relu, model='lstm', num_layers=2)
best_model.eval()
checkpoint = torch.load('movie_model.pth')
best_model.load_state_dict(checkpoint['model_state_dict'])

tensor = torch.tensor(text_vectorDF.values, dtype=torch.float)
# 출력층에서 로짓 값을 반환하므로, sigmoid로 확률을 압축
predicted_value = torch.sigmoid(best_model(tensor.unsqueeze(1)))
predicted_value = (predicted_value > 0.5).float().item()  # 0.5를 기준으로 0 또는 1로 변환

if predicted_value == 0:
    predicted_value = '15세미만관람가'
elif predicted_value == 1:
    predicted_value = '15세이상관람가'

predicted_value

'15세미만관람가'